# 함수화한 모델

In [131]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import warnings

warnings.filterwarnings(action = 'ignore')

## 1. 데이터 로드 및 전처리

### 1.1. 데이터 로드

In [132]:
sample_df = pd.read_csv('final-sample-fix (1).csv',index_col=0)
sample_df.head(3)

,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
0,종로구,1111010100,청운동,0,0,0,0,0,0,0,0,2,1,0,8,0,0,0,1,0
1,종로구,1111010200,신교동,0,0,0,0,0,0,0,0,5,1,0,1,0,0,0,0,0
2,종로구,1111010300,궁정동,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0


In [133]:
sample_df.set_index('DONG_CODE',inplace=True)
sample_df.head(2)

,GU,DONG,SUBWAY_NUM,STARBUCKS_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
DONG_CODE,,,,,,,,,,,,,,,,,,,
1111010100,종로구,청운동,0,0,0,0,0,0,0,0,2,1,0,8,0,0,0,1,0
1111010200,종로구,신교동,0,0,0,0,0,0,0,0,5,1,0,1,0,0,0,0,0


In [134]:
sample_df.shape

(467, 19)

In [152]:
data = sample_df.iloc[:, 2:]
data.tail()

,SUBWAY_NUM,STARBUCKS_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
DONG_CODE,,,,,,,,,,,,,,,,,
1174010700,1,1,2,0,30,4,16,8,197,1,0,70,4,0,11,33,0
1174010900,1,4,3,1,70,6,14,7,124,1,0,131,17,0,12,68,1
1174010800,4,4,2,0,50,2,30,14,190,1,0,143,20,0,12,58,0
1174010500,0,2,1,0,20,1,18,9,100,1,0,63,10,0,10,38,0
1174010600,1,0,0,1,0,0,8,6,76,1,0,35,4,0,0,17,0


In [139]:
pop_df = pd.read_csv('POPU_DONG.csv',index_col=0,encoding='cp949')
pop_df.head(3)

,SI_DO,GU,DONG,NUM_POPU,AREA
DONG_CODE,,,,,
1111010100,서울특별시,종로구,청운동,2330,797032.5
1111010200,서울특별시,종로구,신교동,1393,99144.0
1111010300,서울특별시,종로구,궁정동,179,73042.7


In [140]:
pop_df = pop_df.drop(['SI_DO','GU','DONG'], axis=1)
pop_df['DENSITY'] = pop_df['NUM_POPU'] / pop_df['AREA']
pop_df

,NUM_POPU,AREA,DENSITY
DONG_CODE,,,
1111010100,2330,797032.5,0.002923
1111010200,1393,99144.0,0.014050
1111010300,179,73042.7,0.002451
1111010400,572,59135.9,0.009673
1111010500,439,59706.1,0.007353
...,...,...,...
1174010600,24311,2461511.8,0.009876
1174010700,66490,4711364.2,0.014113
1174010800,65873,1958987.6,0.033626


In [151]:
init_df = pd.merge(sample_df, pop_df,left_index=True, right_index=True)
# a = init_df.iloc[:,2:] / init_df['DENSITY'].values


array([2.92334378e-03, 1.40502703e-02, 2.45062135e-03, 9.67263540e-03,
       7.35268256e-03, 1.50007252e-02, 1.23795089e-02, 1.46025823e-02,
       4.71504018e-03, 0.00000000e+00, 4.93962068e-03, 8.34649065e-04,
       1.61324231e-02, 1.16346916e-02, 1.77957053e-03, 1.16350357e-02,
       3.43080243e-05, 7.11104111e-03, 3.75324076e-02, 1.31066992e-03,
       2.38521448e-03, 6.75243872e-03, 1.09880211e-03, 3.71326546e-03,
       2.33176632e-03, 4.23751349e-03, 4.21999308e-03, 0.00000000e+00,
       2.66136160e-03, 4.46805230e-03, 9.96810814e-03, 1.57598140e-03,
       2.31232493e-03, 2.22200218e-02, 2.72149714e-02, 1.97572749e-02,
       2.04917573e-02, 3.76369460e-02, 3.00703753e-02, 3.31580257e-02,
       6.67934224e-03, 1.59546513e-03, 6.16733144e-03, 1.11910748e-02,
       1.20231559e-03, 2.11634389e-05, 2.43727074e-03, 1.30552138e-04,
       3.30250991e-03, 5.16476904e-05, 1.51279745e-03, 4.91838289e-03,
       1.55485669e-04, 4.12851905e-03, 2.35997483e-03, 2.27965814e-02,
      

### 1.2 유저 데이터 입력 및 스케일링

In [13]:
user = [2,1,3,2,1,4,1,3,4,1,0,0,0,1,0,0,1]

In [14]:
len(user)

17

In [15]:
def user_transform(user): # min_Max Scaling 함수
    user_min = min(user)
    user_max = max(user)
    
    user_scaled = []
    for u in user:
        x = (u - user_min) / (user_max - user_min)
        user_scaled.append(x)
    return user_scaled

In [17]:
user_scaled = user_transform(user)
print(user_scaled)

[0.5, 0.25, 0.75, 0.5, 0.25, 1.0, 0.25, 0.75, 1.0, 0.25, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.25]


### 1.3 가중치 matrix

In [8]:
# 임시 가중치 matrix 생성
weight = [[1,0,0,0,0,0,0.2,0.2,0,0,0,0.5,0,0.3],
          [0,1,0.3,0,0,0,0.7,0,0,0,0,0,0.1,0],
          [0,0,1,0,0,0,0.3,0,0,0,0,0,0,0],
          [0,0,0,1,0,0,0,0,0.2,0,0,0.1,0,0],
          [0,0,0,0,1,0,0,0,0,0,0.6,0,0,0],
          [0,0,0,0,0,1,0,0,0,0,0,0,0,0],
          [0,0,0.6,0,0,0,1,0,0.3,0,0,0,0,0],
          [0,0.2,0,0,0,0,0,1,0,0,0,0.7,0,0],
          [0,0,0,0,0,0,0,0,1,0,0,0.4,0,0],
          [0,0,0,0.3,0,0,0,0,0,1,0,0.2,0,0],
          [0,0,0,0,0.5,0,0,0,0,0,1,0,0,0],
          [0,0,0,0.8,0,0.3,0,0,0,0,0,1,0,0],
          [0,0,0,0,0.6,0,0,0,0.2,0,0,0,1,0],
          [0,0,0.3,0,0,0,0,0.6,0,0,0,0,0,1]]

## 2. 군집화

### 2.1 군집화 및 분류 결과 확인

In [24]:
kmeans = KMeans(n_clusters=3, init='k-means++',max_iter=300,random_state=0)
kmeans.fit(data)

KMeans(n_clusters=3, random_state=0)

In [25]:
kmeans.labels_
df = sample_df.copy()
df['km_cluster'] = kmeans.labels_
df.groupby('km_cluster').count()

,GU,DONG,SUBWAY_NUM,STARBUCKS_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
km_cluster,,,,,,,,,,,,,,,,,,,
0,396,396,396,396,396,396,396,396,396,396,396,396,396,396,396,396,396,396,396
1,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
2,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68


### 2.2 사용자 분류

In [18]:
def user(user_scaled,user_name):
    user_df = pd.DataFrame(columns=col,index=[user_name])
    user_df.loc[user_name] = user_scaled
    cluster_user = kmeans.predict(user_df)
    user_df['km_cluster'] = cluster_user
    return user_df

In [19]:
user_df = user(user_scaled,'user')
user_df

,A,B,C,D,E,a,b,c,d,e,f,g,h,i,km_cluster
user,0.75,0.125,1.0,0.5,1.0,0.125,0.0,0.125,0.0,0.125,0.125,0.0,0.125,0.125,1


In [20]:
# 유저의 군집에 해당하는 지역 추출 함수
def recommand_area(df, user_df, user_name): # 지역 데이터, 유저 데이터, 유저 이름
    rc_area = user_df['km_cluster'].loc[user_name] # 해당 유저의 군집
    rc_seoul = df[df['km_cluster'] == rc_area] # 서울시 내 군집 지역 추출
    rc_list = rc_seoul.index.tolist() # 추천 리스트
    return rc_seoul, rc_list

In [21]:
rc_seoul = recommand_area(df,user_df,'user')[0]
rc_list = recommand_area(df,user_df,'user')[1] 

In [22]:
rc_list # 'user'가 해당하는 군집에 속한 서울 동 리스트

['삼청동',
 '혜화동',
 '명동',
 '필동',
 '약수동',
 '동화동',
 '이태원2동',
 '원효로1동',
 '성수1가2동',
 '능동',
 '청량리동',
 '중화2동',
 '면목3.8동',
 '인수동',
 '창3동',
 '도봉2동',
 '월계2동',
 '중계1동',
 '녹번동',
 '대조동',
 '응암2동',
 '천연동',
 '홍제3동',
 '남가좌1동',
 '용강동',
 '대흥동',
 '아현동',
 '신월3동',
 '신월6동',
 '등촌1동',
 '공항동',
 '방화1동',
 '구로2동',
 '개봉2동',
 '가리봉동',
 '당산2동',
 '신길3동',
 '신길5동',
 '신길6동',
 '대림3동',
 '영등포본동',
 '상도4동',
 '신대방2동',
 '사당2동',
 '청림동',
 '낙성대동',
 '중앙동',
 '조원동',
 '내곡동',
 '대치4동',
 '도곡2동',
 '일원본동',
 '압구정동',
 '청담동',
 '오륜동',
 '장지동',
 '잠실3동',
 '잠실4동',
 '길동',
 '강일동',
 '암사2동',
 '성내2동']

### 2.3 유사도 측정

In [39]:
def similarity(user_df, df, user_name, num): # 유저 데이터, 유사도 측정을 위한 데이터, 유저 이름, 원하는 순위
    con_data = pd.concat([user_df.loc[[user_name]],df])
    rc_sim = cosine_similarity(con_data,con_data)
    sim_matrix = pd.DataFrame(rc_sim,columns=con_data.index).loc[[0]].T
    rank = sim_matrix[0].sort_values(ascending=False) # 유사도 순서로 정렬
    ranking = rank[1:num+1].index.tolist() # 1~n 위 리스트
    return ranking

In [42]:
ranking = similarity(user_df,df,'user',4)
cluster_rank = similarity(user_df,rc_seoul,'user',4)

**겹치는 지역 제거**

In [81]:
def concat_df(rank_1,rank_2): # 두 집단 병합 및 겹침 제거 함수
    tmp_1 = df.loc[rank_1]
    tmp_2 = df.loc[rank_2]
    x = pd.concat([tmp_1,tmp_2])
    x.reset_index(inplace=True)
    x = x.drop_duplicates(['동이름'])
    x.loc[:4]
    x.set_index('동이름',inplace=True)
    del(x['km_cluster'])
    return x

In [82]:
con_df = concat_df(ranking,cluster_rank)
con_df

,A,B,C,D,E,a,b,c,d,e,f,g,h,i
동이름,,,,,,,,,,,,,,
청림동,0.9,0.2,0.9,0.7,0.9,0.5,0.4,0.4,0.4,0.6,0.4,0.3,0.1,0.8
개봉2동,1.0,0.3,1.0,0.7,0.8,0.7,0.1,0.3,0.8,0.3,0.3,0.4,0.3,0.4
대치4동,0.6,0.5,0.5,0.2,1.0,0.2,0.5,0.2,0.1,0.6,0.1,0.2,0.2,0.1
월계2동,0.9,0.1,1.0,0.1,0.8,0.5,0.1,0.4,0.3,0.6,0.9,0.5,0.3,0.2


In [90]:
def weighting(df, weight): # 가중치 부여 된 데이터 프레임 생성 함수
    data = df.values
    w = [0] * 14
    for i in range(len(weight)):
        if(user_df.loc['user'][i] != 0):
            for k in range(len(weight)):
                w[i] += weight[i][k]
    modified = []
    for k in range(len(data)):
        modi = []
        for i in range(len(w)):
            mod = data[k][i] * (1 + w[i])
            modi.append(mod)
        modified.append(modi)
    weighted_df = pd.DataFrame(modified,index=df.index,columns=col)
    return weighted_df

In [107]:
weighted_df = weighting(con_df,weight)
weighted_df

,A,B,C,D,E,a,b,c,d,e,f,g,h,i
동이름,,,,,,,,,,,,,,
청림동,2.88,0.62,2.07,1.61,2.34,1.0,0.4,1.16,0.4,1.50,1.00,0.3,0.28,2.32
개봉2동,3.20,0.93,2.30,1.61,2.08,1.4,0.1,0.87,0.8,0.75,0.75,0.4,0.84,1.16
대치4동,1.92,1.55,1.15,0.46,2.60,0.4,0.5,0.58,0.1,1.50,0.25,0.2,0.56,0.29
월계2동,2.88,0.31,2.30,0.23,2.08,1.0,0.1,1.16,0.3,1.50,2.25,0.5,0.84,0.58


In [133]:
def ranking(weighted_df): # 가중치 데이터 프레임으로 3위까지
    sums = []
    for i in range(len(weighted_df.index)):
        sums.append(weighted_df.iloc[i].sum())
    weighted_df['sum'] = sums
    oh = weighted_df.T.loc['sum'].sort_values(ascending=False)
    x = pd.DataFrame(oh).index.tolist()[0:3]
    return x

In [135]:
x = ranking(weighted_df)
x

['청림동', '개봉2동', '월계2동']